In [4]:
#imports
import pickle as pic
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import add
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import Sequence

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [5]:
#residual module
def res_module(inp):
    if not inp.shape[-1] == 64:
        #project with 1x1 convolution
        inp = Conv2D(64,1,activation="relu")(inp)
    x = BatchNormalization()(inp)
    x = Activation("relu")(x)
    x = Conv2D(16, 3,padding="same",activation="sigmoid")(x)
    x = Conv2D(32, 3,padding="same",activation="sigmoid")(x)
    x = Conv2D(64, 3,padding="same",activation="sigmoid")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(16, 3,padding="same",activation="sigmoid")(x)
    x = Conv2D(32, 3,padding="same",activation="sigmoid")(x)
    x = Conv2D(64, 3,padding="same",activation="sigmoid")(x)
    out = add([x, inp])
    return out

In [7]:
#extracting output tensor for given extraction cordinates
def extract_tensor(inp):
    return tf.slice(inp[0], inp[1], [5, 5, 64]) #crop 5 * 5 tensor out 
    #inp[0] encoded image, inp[1] crop tensor


In [13]:
lG_inp = Input(shape = (95, 65, 3))

ext_inp = Input(shape = (3), dtype = tf.int32)

#defining model

#four residual module stacked 
x = res_module(lG_inp)
x = res_module(x)
x = res_module(x)
x = res_module(x)

# now x is 95 * 65 * 54 res encoded tensor, now carry out extraction procedure to enforce localization
# batch_size is Dynamic, Unknown or of type None, hence using map_fn to iterate over dimention '0' --> None

extracted_tensor = tf.map_fn(lambda x : tf.slice(x[0], x[1], [5,5,64]), elems = (x, ext_inp), dtype = (tf.float32))

x = Flatten()(extracted_tensor) #flatten and feed to dense layer

#fully connected layer 1

x1 = Dense(128, activation='relu')(x)

#fully connected layer 2
x2 = Dense(1, activation='sigmoid')(x1)

#fully connected layer 3
x3 = Dense(25, activation='sigmoid')(x1)

x3 = tf.reshape(x3, (5,5)) #output a 5 * 5 image

model = Model(inputs= [ lG_inp, ext_inp], outputs= [x2,x3])

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 95, 65, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_100 (Conv2D)             (None, 95, 65, 64)   256         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_32 (BatchNo (None, 95, 65, 64)   256         conv2d_100[0][0]                 
__________________________________________________________________________________________________
activation_32 (Activation)      (None, 95, 65, 64)   0           batch_normalization_32[0][0]     
______________________________________________________________________________________________